# RNNMorhp Predictor for text analysis

**Ссылка**, на источник текста

In [56]:
DATA_URL = 'http://az.lib.ru/g/gogolx_n_w/text_0050.shtml'
# DATA_URL = 'http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml'

Устанавливаем библиотеки

In [57]:
import warnings
warnings.filterwarnings('ignore')

Создаём объект морфологического анализатора `RNNMorph`

In [58]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [59]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [60]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Гоголь Николай Васильевич. Вий</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/g/gogolx_n_w/>Гоголь Николай Васильевич</a><br>\r\nВий</h2>\r\n\r\n<!------- П'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [61]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [62]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Гоголь Николай Васильевич. Вий\n\n\n\nГоголь Николай Васильевич\r\nВий\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n\n\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [63]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mikhail.komarov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 934 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [64]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent] 
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться

15/15 [==============================] - 4s 182ms/step


sentences: 100%|██████████| 934/934 [00:00<00:00, 233600.47it/s]


[['пфейфер', '(', 'он', '.', ')']]

In [65]:
predictions = list(list(filter(lambda x: str.isalpha(x), i)) for i in predictions)

Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [66]:
len(predictions)

934

In [67]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)

11732

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам. 

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75. 

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [68]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mikhail.komarov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [69]:
fdist = FreqDist(non_uniq_tokens)

In [70]:
N = 150

top_N_freq_tokens = list(map(lambda x: x[0], fdist.most_common(N)))

len(list(filter(lambda x: x not in STOPWORDS, top_N_freq_tokens)))/N

0.5733333333333334

Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [71]:
len(list(filter(lambda x: x[1] > 10, fdist.most_common())))

161

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
